In [4]:
!pip install azure-eventhub

In [5]:
import pandas as pd
import random
from datetime import datetime
 
# Step 1: Define product types and unique product names
product_types = ["Footwear", "Headgear", "BaseLayer"]
 
product_type_mapping = {
    "Footwear": [
        "TrailRunner", "FlexShoe", "HeatGear", "ProTrainer", "WinterBoots",
        "ActiveCap", "UrbanFit", "SprintVest", "ThermalShirt", "AlpineHelmet"
    ],
    "Headgear": [
        "ActiveCap", "ThermalShirt", "AlpineHelmet", "UrbanFit", "TrailRunner",
        "HeatGear", "SprintVest", "FlexShoe", "WinterBoots"
    ],
    "BaseLayer": [
        "SprintVest", "ThermalShirt", "HeatGear", "ActiveCap", "UrbanFit",
        "AlpineHelmet", "FlexShoe", "WinterBoots", "TrailRunner"
    ]
}
 
# Get all unique product names
product_names = set()
for names in product_type_mapping.values():
    product_names.update(names)
 
# Step 2: All (productName, productType) combinations
all_combinations = [(name, ptype) for name in product_names for ptype in product_types]
 
# Step 3: Pad to 89 if needed
if len(all_combinations) < 89:
    extended_combinations = all_combinations + random.choices(all_combinations, k=89 - len(all_combinations))
else:
    extended_combinations = all_combinations
 
random.shuffle(extended_combinations)
selected_combinations = extended_combinations[:89]
 
# Step 4: Generate inventory
def generate_inventory_and_status():
    inv = random.randint(0, 100)
    if inv == 0:
        return inv, "Out of Stock"
    elif inv <= 15:
        return inv, "Low Stock"
    elif inv <= 50:
        return inv, "Medium Stock"
    else:
        return inv, "In Stock"
 
# Step 5: Generate records
records = []
material_counter = 1
 
for i, (product_name, product_type) in enumerate(selected_combinations):
    product_id = f"P{str(i).zfill(5)}"
 
    # Assign same materialId for every 3 records
    if i % 3 == 0:
        current_material_id = f"M{str(material_counter).zfill(5)}"
        material_counter += 1
 
    design_id = f"Design_{random.randint(1000, 9999)}"
    inventory_sku, stock_status = generate_inventory_and_status()
 
    records.append({
        "productId": product_id,
        "productName": product_name,
        "productType": product_type,
        "designId": design_id,
        "materialId": current_material_id,
        "inventorySKU": inventory_sku,
        "stockStatus": stock_status
    })
 
# Step 6: Create and display DataFrame
df_apparel = pd.DataFrame(records)
display(df_apparel)
 
# Optional: Write to Spark table
# df_spark = spark.createDataFrame(df_apparel)
# df_spark.write.mode("append").saveAsTable("apparel_product_data")

In [ ]:
from azure.eventhub import EventHubProducerClient, EventData

# Event Hub configuration
CONNECTION_STR = ""

EVENT_HUB_NAME = "es_921a7307-7c59-44ea-9840-892268cce061"

producer = EventHubProducerClient.from_connection_string(conn_str=CONNECTION_STR, eventhub_name=EVENT_HUB_NAME)

# Start with a new batch
batch = producer.create_batch()

# Send records from the DataFrame
for _, row in df_apparel.iterrows(): 
    event_json = row.to_json()
    try:
        batch.add(EventData(event_json))
    except ValueError:
        # Batch full → send and start a new one
        producer.send_batch(batch)
        batch = producer.create_batch()
        batch.add(EventData(event_json))

# Send remaining records
producer.send_batch(batch)
producer.close()

print("✅ All events sent to Eventstream.")
